In [9]:
# Initial importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.feature_selection import f_regression, SelectKBest, f_classif
from sklearn.linear_model import Ridge, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier, XGBRegressor

In [3]:
# Enabling the entire df to be viewed when it goes beyond the normal 80 cols/rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

In [4]:
# Reading in the csv for the training features
df = pd.read_csv('train_features.csv')
df.head().T

,0,1,2,3,4
id,69572,8776,34310,67743,19728
amount_tsh,6000,0,25,0,0
date_recorded,2011-03-14,2013-03-06,2013-02-25,2013-01-28,2011-07-13
funder,Roman,Grumeti,Lottery Club,Unicef,Action In A
gps_height,1390,1399,686,263,0
installer,Roman,GRUMETI,World vision,UNICEF,Artisan
longitude,34.9381,34.6988,37.4607,38.4862,31.1308
latitude,-9.85632,-2.14747,-3.82133,-11.1553,-1.82536
wpt_name,none,Zahanati,Kwa Mahundi,Zahanati Ya Nanyumbu,Shuleni
num_private,0,0,0,0,0


In [5]:
# Reading in the csv for the training labels
df_labels = pd.read_csv('train_labels.csv')
df_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [6]:
# Reading in the csv for the test data
df_test = pd.read_csv('test_features.csv')
df_test.head().T

,0,1,2,3,4
id,50785,51630,17168,45559,49871
amount_tsh,0,0,0,0,500
date_recorded,2013-02-04,2013-02-04,2013-02-01,2013-01-22,2013-03-27
funder,Dmdd,Government Of Tanzania,NaN,Finn Water,Bruder
gps_height,1996,1569,1567,267,1260
installer,DMDD,DWE,NaN,FINN WATER,BRUDER
longitude,35.2908,36.6567,34.7679,38.058,35.0061
latitude,-4.0597,-3.30921,-5.00434,-9.41867,-10.9504
wpt_name,Dinamu Secondary School,Kimnyak,Puma Secondary,Kwa Mzee Pange,Kwa Mzee Turuka
num_private,0,0,0,0,0


In [7]:
# verifying shape of these dataframes
df.shape, df_labels.shape, df_test.shape

((59400, 40), (59400, 2), (14358, 40))

In [9]:
# Verifying the value counts of the target value, status_group
df_labels.status_group.value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

So we would expect the following percentages for these outcomes: functional: 54.3% non functional 38.4% functional needs repair 7.3%

In [ ]:
# Verifying the data types of each column in our dataframe
df_merged.dtypes

In [10]:
# Just selecting the numeric features from the features dataframe
df_num = df.select_dtypes(include=['number'])
print(df_num.shape)
df_num.head()

(59400, 10)


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
1,8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009
3,67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986
4,19728,0.0,0,31.130847,-1.825359,0,18,1,0,0


In [18]:
# Just selecting the numeric features from the test dataframe
df_num_test = df_test.select_dtypes(include=['number'])
print(df_num_test.shape)
df_num_test.head()

(14358, 10)


,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
0,50785,0.0,1996,35.290799,-4.059696,0,21,3,321,2012
1,51630,0.0,1569,36.656709,-3.309214,0,2,2,300,2000
2,17168,0.0,1567,34.767863,-5.004344,0,13,2,500,2010
3,45559,0.0,267,38.058046,-9.418672,0,80,43,250,1987
4,49871,500.0,1260,35.006123,-10.950412,0,10,3,60,2000


In [16]:
# Initializing our X and y variables for regression 
X = df_num
y = df_labels.status_group

In [17]:
# Using Logistic Regression model to make baseline prediction
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)
y_pred = model.predict(X)
print(classification_report(y, y_pred))
print('accuracy', accuracy_score(y, y_pred))

C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\metric

                         precision    recall  f1-score   support

             functional       0.56      0.93      0.70     32259
functional needs repair       0.00      0.00      0.00      4317
         non functional       0.49      0.12      0.19     22824

              micro avg       0.55      0.55      0.55     59400
              macro avg       0.35      0.35      0.30     59400
           weighted avg       0.49      0.55      0.45     59400

accuracy 0.5518013468013468


In [22]:
# Fitting and scoring our Logistic Regression model
log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5517845117845118

In [26]:
# Predicting outcomes on our test data
pred_test = log_reg.predict(df_num_test)

In [27]:
# creating a dataframe for the predicted outcomes
pred = pd.DataFrame(pred_test)
pred.head()

,0
0,functional
1,functional
2,functional
3,non functional
4,functional


In [28]:
# Adding the 'id' column to the prediction dataframe
pred = pd.concat([df_num_test.id, pred], axis=1)
pred.head()

,id,0
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [29]:
# Remaning the status_group column to the correct name
pred.columns.values[1]= 'status_group'
pred.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [30]:
# Verifying the dataframe has the correct shape for submission
pred.shape

,id
count,14358.000000
mean,37232.859799
std,21382.890432
min,10.000000
25%,18765.500000
50%,37442.000000
75%,55909.250000
max,74249.000000


In [31]:
# ***PRINT TO CSV***
pred.to_csv('pred_test.csv', sep=',', encoding='utf-8', index=False)
# Or use this to print to csv
#pd.DataFrame(pred).to_csv("submission_pd.csv", index = False)

First submission and score! 

In [38]:
# Verifying the shape of features to see which has too many unique values to use in onehot encoding
# Must find a better way to do this part when I have time...
df.amount_tsh.value_counts().shape

(98,)

In [39]:
df.date_recorded.value_counts().shape

(356,)

In [40]:
df.funder.value_counts().shape

(1897,)

In [41]:
df.installer.value_counts().shape

(2145,)

In [42]:
df.longitude.value_counts().shape

(57516,)

In [43]:
df.latitude.value_counts().shape

(57517,)

In [44]:
df.wpt_name.value_counts().shape

(37400,)

In [45]:
df.num_private.value_counts().shape

(65,)

In [46]:
df.basin.value_counts().shape

(9,)

In [47]:
df.subvillage.value_counts().shape

(19287,)

In [48]:
df.region.value_counts().shape

(21,)

In [49]:
df.region_code.value_counts().shape

(27,)

In [50]:
df.district_code.value_counts().shape

(20,)

In [51]:
df.lga.value_counts().shape

(125,)

In [52]:
df.ward.value_counts().shape

(2092,)

In [53]:
df.population.value_counts().shape

(1049,)

In [54]:
df.public_meeting.value_counts().shape

(2,)

In [55]:
df.recorded_by.value_counts().shape

(1,)

In [56]:
df.scheme_management.value_counts().shape

(12,)

In [57]:
df.scheme_name.value_counts().shape

(2696,)

In [58]:
df.permit.value_counts().shape

(2,)

In [59]:
df.construction_year.value_counts().shape

(55,)

In [60]:
df.extraction_type.value_counts().shape

(18,)

In [62]:
df.extraction_type_group.value_counts().shape

(13,)

In [63]:
df.extraction_type_class.value_counts().shape

(7,)

In [64]:
df.management.value_counts().shape

(12,)

In [65]:
df.management_group.value_counts().shape

(5,)

In [66]:
df.payment.value_counts().shape

(7,)

In [67]:
df.payment_type.value_counts().shape

(7,)

In [68]:
df.water_quality.value_counts().shape

(8,)

In [69]:
df.quality_group.value_counts().shape

(6,)

In [70]:
df.quantity.value_counts().shape

(5,)

In [71]:
df.quantity_group.value_counts().shape

(5,)

In [72]:
df.source.value_counts().shape

(10,)

In [73]:
df.source_type.value_counts().shape

(7,)

In [74]:
df.source_class.value_counts().shape

(3,)

In [75]:
df.waterpoint_type.value_counts().shape

(7,)

In [76]:
df.waterpoint_type_group.value_counts().shape

(6,)

In [77]:
# verifying the numeric columns
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [126]:
# Dropping columns I dont find useful(too many unique values/odd observations)
df_drop = df.drop(columns=['id', 'longitude', 'latitude', 'num_private', 
                            'date_recorded', 'funder', 'installer', 'wpt_name', 
                            'lga', 'ward', 'recorded_by', 'scheme_name', 
                            'subvillage', 'public_meeting', 'scheme_management', 'permit',
                            'funder', 'installer', 'scheme_name'])
  
df_drop.head()

,amount_tsh,gps_height,basin,region,region_code,district_code,population,construction_year,extraction_type,extraction_type_group,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,6000.0,1390,Lake Nyasa,Iringa,11,5,109,1999,gravity,gravity,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,0.0,1399,Lake Victoria,Mara,20,2,280,2010,gravity,gravity,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,25.0,686,Pangani,Manyara,21,4,250,2009,gravity,gravity,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,0.0,263,Ruvuma / Southern Coast,Mtwara,90,63,58,1986,submersible,submersible,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,0.0,0,Lake Victoria,Kagera,18,1,0,0,gravity,gravity,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [90]:
# Verifying nulls
df_drop.isnull().sum()

amount_tsh               0
gps_height               0
basin                    0
region                   0
region_code              0
district_code            0
population               0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [116]:
# Another null check
def no_nulls(df):
    return not any(df.isnull().sum())

no_nulls(df_drop)

True

In [120]:
# Verifying head of dataframe after dropping columns
df_drop.head()

,amount_tsh,gps_height,basin,region,region_code,district_code,population,construction_year,extraction_type,extraction_type_group,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,6000.0,1390,Lake Nyasa,Iringa,11,5,109,1999,gravity,gravity,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,0.0,1399,Lake Victoria,Mara,20,2,280,2010,gravity,gravity,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,25.0,686,Pangani,Manyara,21,4,250,2009,gravity,gravity,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,0.0,263,Ruvuma / Southern Coast,Mtwara,90,63,58,1986,submersible,submersible,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,0.0,0,Lake Victoria,Kagera,18,1,0,0,gravity,gravity,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [93]:
# Verifying numeric features
df_drop.describe()

,amount_tsh,gps_height,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,15.297003,5.629747,179.909983,1300.652475
std,2997.574558,693.116350,17.587406,9.633649,471.482176,951.620547
min,0.000000,-90.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,369.000000,12.000000,3.000000,25.000000,1986.000000
75%,20.000000,1319.250000,17.000000,5.000000,215.000000,2004.000000
max,350000.000000,2770.000000,99.000000,80.000000,30500.000000,2013.000000


In [81]:
# I had to install category_encoders locally, but will not need to run this again
#!pip install category_encoders

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [82]:
# Importing category_ecnoders for onehot encoding
import category_encoders as ce

In [108]:
# Verifying which columns are numeric and which are strings/objects
df_drop.dtypes

amount_tsh               float64
gps_height                 int64
basin                     object
region                    object
region_code                int64
district_code              int64
population                 int64
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_quality             object
quality_group             object
quantity                  object
quantity_group            object
source                    object
source_type               object
source_class              object
waterpoint_type           object
waterpoint_type_group     object
dtype: object

In [118]:
# testing one hot on full df
#one_hot_test = pd.get_dummies(data=df_drop)
#one_hot_test.head()

,amount_tsh,gps_height,region_code,district_code,population,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,6000.0,1390,11,5,109,1999,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,1399,20,2,280,2010,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,25.0,686,21,4,250,2009,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.0,263,90,63,58,1986,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.0,0,18,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [127]:
# 1hot encode non-numeric columns
df_one_hot = pd.get_dummies(df_drop)
df_one_hot.head()

,amount_tsh,gps_height,region_code,district_code,population,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,6000.0,1390,11,5,109,1999,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,1399,20,2,280,2010,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,25.0,686,21,4,250,2009,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0.0,263,90,63,58,1986,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.0,0,18,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [148]:
# Dropping same columns on the test dataframe
df_drop_test = df_test.drop(columns=['id', 'longitude', 'latitude', 'num_private', 
                            'date_recorded', 'funder', 'installer', 'wpt_name', 
                            'lga', 'ward', 'recorded_by', 'scheme_name', 
                            'subvillage', 'public_meeting', 'scheme_management', 'permit',
                            'funder', 'installer', 'scheme_name'])
                           
df_drop_test.head()

,amount_tsh,gps_height,basin,region,region_code,district_code,population,construction_year,extraction_type,extraction_type_group,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,0.0,1996,Internal,Manyara,21,3,321,2012,other,other,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,0.0,1569,Pangani,Arusha,2,2,300,2000,gravity,gravity,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,0.0,1567,Internal,Singida,13,2,500,2010,other,other,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,0.0,267,Ruvuma / Southern Coast,Lindi,80,43,250,1987,other,other,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,500.0,1260,Ruvuma / Southern Coast,Ruvuma,10,3,60,2000,gravity,gravity,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [149]:
# Null verification
df_drop_test.isnull().sum()

amount_tsh               0
gps_height               0
basin                    0
region                   0
region_code              0
district_code            0
population               0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [150]:
# More null verification
def no_nulls(df):
    return not any(df.isnull().sum())

no_nulls(df_drop_test)

True

In [151]:
# checking numeric features
df_drop_test.describe()

,amount_tsh,gps_height,region_code,district_code,population,construction_year
count,14358.000000,14358.0000,14358.000000,14358.000000,14358.000000,14358.000000
mean,324.219996,653.6363,15.156359,5.713052,187.055439,1298.251985
std,2533.367778,688.2721,17.387588,9.794304,476.065978,952.551852
min,0.000000,-57.0000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.0000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,346.0000,12.000000,3.000000,25.000000,1986.000000
75%,25.000000,1306.0000,17.000000,5.000000,230.000000,2004.000000
max,200000.000000,2777.0000,99.000000,80.000000,11469.000000,2013.000000


In [152]:
# Verifying shape to ensure we have correct feature and observation counts
df_drop_test.shape

(14358, 24)

In [153]:
# 1hot encode non-numeric columns for test data
df_one_hot_test = pd.get_dummies(df_drop_test)
df_one_hot_test.head()

,amount_tsh,gps_height,region_code,district_code,population,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,0.0,1996,21,3,321,2012,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,0.0,1569,2,2,300,2000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0.0,1567,13,2,500,2010,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0.0,267,80,43,250,1987,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,500.0,1260,10,3,60,2000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [155]:
# Swapping '/' characters for '_' from data as it was causing errors, in test dataframe
df_one_hot_test.columns = [x.strip().replace('/', '_') for x in df_one_hot_test.columns]

df_one_hot_test.head().T

,0,1,2,3,4
amount_tsh,0.0,0.0,0.0,0.0,500.0
gps_height,1996.0,1569.0,1567.0,267.0,1260.0
region_code,21.0,2.0,13.0,80.0,10.0
district_code,3.0,2.0,2.0,43.0,3.0
population,321.0,300.0,500.0,250.0,60.0
construction_year,2012.0,2000.0,2010.0,1987.0,2000.0
basin_Internal,1.0,0.0,1.0,0.0,0.0
basin_Lake Nyasa,0.0,0.0,0.0,0.0,0.0
basin_Lake Rukwa,0.0,0.0,0.0,0.0,0.0
basin_Lake Tanganyika,0.0,0.0,0.0,0.0,0.0


In [131]:
# Checking head after one hot encoding
df_one_hot.head().T

,0,1,2,3,4
amount_tsh,6000.0,0.0,25.0,0.0,0.0
gps_height,1390.0,1399.0,686.0,263.0,0.0
region_code,11.0,20.0,21.0,90.0,18.0
district_code,5.0,2.0,4.0,63.0,1.0
population,109.0,280.0,250.0,58.0,0.0
construction_year,1999.0,2010.0,2009.0,1986.0,0.0
basin_Internal,0.0,0.0,0.0,0.0,0.0
basin_Lake Nyasa,1.0,0.0,0.0,0.0,0.0
basin_Lake Rukwa,0.0,0.0,0.0,0.0,0.0
basin_Lake Tanganyika,0.0,0.0,0.0,0.0,0.0


In [129]:
# Checking numeric cols after one hot
df_one_hot.describe()

,amount_tsh,gps_height,region_code,district_code,population,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
count,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,...,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,15.297003,5.629747,179.909983,1300.652475,0.131061,0.085606,0.041313,0.108283,...,0.000118,0.294411,0.013199,0.107407,0.001953,0.582912,0.000118,0.294411,0.013199,0.107407
std,2997.574558,693.116350,17.587406,9.633649,471.482176,951.620547,0.337469,0.279784,0.199015,0.310740,...,0.010855,0.455781,0.114126,0.309633,0.044148,0.493082,0.010855,0.455781,0.114126,0.309633
min,0.000000,-90.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,369.000000,12.000000,3.000000,25.000000,1986.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,20.000000,1319.250000,17.000000,5.000000,215.000000,2004.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,350000.000000,2770.000000,99.000000,80.000000,30500.000000,2013.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [132]:
# Fixing '/' to '_', this time on train dataframe
df_one_hot.columns = [x.strip().replace('/', '_') for x in df_one_hot.columns]
df_one_hot.head().T

,0,1,2,3,4
amount_tsh,6000.0,0.0,25.0,0.0,0.0
gps_height,1390.0,1399.0,686.0,263.0,0.0
region_code,11.0,20.0,21.0,90.0,18.0
district_code,5.0,2.0,4.0,63.0,1.0
population,109.0,280.0,250.0,58.0,0.0
construction_year,1999.0,2010.0,2009.0,1986.0,0.0
basin_Internal,0.0,0.0,0.0,0.0,0.0
basin_Lake Nyasa,1.0,0.0,0.0,0.0,0.0
basin_Lake Rukwa,0.0,0.0,0.0,0.0,0.0
basin_Lake Tanganyika,0.0,0.0,0.0,0.0,0.0


In [164]:
# Had one column in train that was not in test so I had to drop it from train dataframe so 
# the number of features in test and train match
df_one_hot = df_one_hot.drop(columns=['extraction_type_other - mkulima_shinyanga'])
df_one_hot.head()

,amount_tsh,gps_height,region_code,district_code,population,construction_year,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma _ Southern Coast,basin_Wami _ Ruvu,region_Arusha,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,extraction_type_afridev,extraction_type_cemo,extraction_type_climax,extraction_type_gravity,extraction_type_india mark ii,extraction_type_india mark iii,extraction_type_ksb,extraction_type_mono,extraction_type_nira_tanira,extraction_type_other,extraction_type_other - play pump,extraction_type_other - rope pump,extraction_type_other - swn 81,extraction_type_submersible,extraction_type_swn 80,extraction_type_walimi,extraction_type_windmill,extraction_type_group_afridev,extraction_type_group_gravity,extraction_type_group_india mark ii,extraction_type_group_india mark iii,extraction_type_group_mono,extraction_type_group_nira_tanira,extraction_type_group_other,extraction_type_group_other handpump,extraction_type_group_other motorpump,extraction_type_group_rope pump,extraction_type_group_submersible,extraction_type_group_swn 80,extraction_type_group_wind-powered,extraction_type_class_gravity,extraction_type_class_handpump,extraction_type_class_motorpump,extraction_type_class_other,extraction_type_class_rope pump,extraction_type_class_submersible,extraction_type_class_wind-powered,management_company,management_other,management_other - school,management_parastatal,management_private operator,management_trust,management_unknown,management_vwc,management_water authority,management_water board,management_wua,management_wug,management_group_commercial,management_group_other,management_group_parastatal,management_group_unknown,management_group_user-group,payment_never pay,payment_other,payment_pay annually,payment_pay monthly,payment_pay per bucket,payment_pay when scheme fails,payment_unknown,payment_type_annually,payment_type_monthly,payment_type_never pay,payment_type_on failure,payment_type_other,payment_type_per bucket,payment_type_unknown,water_quality_coloured,water_quality_fluoride,water_quality_fluoride abandoned,water_quality_milky,water_quality_salty,water_quality_salty abandoned,water_quality_soft,water_quality_unknown,quality_group_colored,quality_group_fluoride,quality_group_good,quality_group_milky,quality_group_salty,quality_group_unknown,quantity_dry,quantity_enough,quantity_insufficient,quantity_seasonal,quantity_unknown,quantity_group_dry,quantity_group_enough,quantity_group_insufficient,quantity_group_seasonal,quantity_group_unknown,source_dam,source_hand dtw,source_lake,source_machine dbh,source_other,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,source_type_borehole,source_type_dam,source_type_other,source_type_rainwater harvesting,source_type_river_lake,source_type_shallow well,source_type_spring,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,6000.0,1390,11,5,109,1999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,0.0,1399,20,2,280,2010,0,0,0,0,1,0,0,

In [165]:
# Setting X and y variables
X = df_one_hot
y = df_labels.status_group

In [166]:
# Fit Logistic Regression model on X and y
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)
y_pred = model.predict(X)
print(classification_report(y, y_pred))
print('accuracy', accuracy_score(y, y_pred))

C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


                         precision    recall  f1-score   support

             functional       0.67      0.90      0.77     32259
functional needs repair       0.00      0.00      0.00      4317
         non functional       0.76      0.52      0.62     22824

              micro avg       0.69      0.69      0.69     59400
              macro avg       0.47      0.47      0.46     59400
           weighted avg       0.65      0.69      0.65     59400

accuracy 0.6903030303030303


In [167]:
# Fit and score Logistic Regression model
log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7338215488215488

Big score improvement over initial Logistic Regression score

In [168]:
# Scoring prediction with test data 
pred_test = log_reg.predict(df_one_hot_test)

In [169]:
# Sending prediction to a dataframe and verifying correct shape
pred = pd.DataFrame(pred_test)
pred.shape

(14358, 1)

In [171]:
# Adding 'id' column to dataframe
pred = pd.concat([df_test.id, pred], axis=1)
pred.head()

,id,0
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [172]:
# Renaming status_group column
pred.columns.values[1]= 'status_group'
pred.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [173]:
# Verifying data looks to be in acceptable ranges
pred.describe()

,id
count,14358.000000
mean,37232.859799
std,21382.890432
min,10.000000
25%,18765.500000
50%,37442.000000
75%,55909.250000
max,74249.000000


In [174]:
# ***PRINT TO CSV***
pred.to_csv('pred_test_again.csv', sep=',', encoding='utf-8', index=False)

Submission number 2 was a success! 

In [10]:
# Reread in data to have fresh dataframes
df_train = pd.read_csv('train_features.csv')
df_labels = pd.read_csv('train_labels.csv')
df_test = pd.read_csv('test_features.csv')

In [11]:
# Verify head of training data
df_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [12]:
# verify head of test data
df_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [13]:
# verify head of feature labels
df_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [14]:
# Checking shape of each dataframe
df_train.shape, df_labels.shape, df_test.shape

((59400, 40), (59400, 2), (14358, 40))

In [15]:
# merge train and labels datasets
df_merged = pd.merge(df_train, df_labels, on='id')
df_merged.head().T

,0,1,2,3,4
id,69572,8776,34310,67743,19728
amount_tsh,6000,0,25,0,0
date_recorded,2011-03-14,2013-03-06,2013-02-25,2013-01-28,2011-07-13
funder,Roman,Grumeti,Lottery Club,Unicef,Action In A
gps_height,1390,1399,686,263,0
installer,Roman,GRUMETI,World vision,UNICEF,Artisan
longitude,34.9381,34.6988,37.4607,38.4862,31.1308
latitude,-9.85632,-2.14747,-3.82133,-11.1553,-1.82536
wpt_name,none,Zahanati,Kwa Mahundi,Zahanati Ya Nanyumbu,Shuleni
num_private,0,0,0,0,0


In [16]:
# set terget 
target=df_merged.pop('status_group')

In [17]:
# Verify head of target
target.head()

0        functional
1        functional
2        functional
3    non functional
4        functional
Name: status_group, dtype: object

In [18]:
# Verify head of merged sataframe
df_merged.head().T

,0,1,2,3,4
id,69572,8776,34310,67743,19728
amount_tsh,6000,0,25,0,0
date_recorded,2011-03-14,2013-03-06,2013-02-25,2013-01-28,2011-07-13
funder,Roman,Grumeti,Lottery Club,Unicef,Action In A
gps_height,1390,1399,686,263,0
installer,Roman,GRUMETI,World vision,UNICEF,Artisan
longitude,34.9381,34.6988,37.4607,38.4862,31.1308
latitude,-9.85632,-2.14747,-3.82133,-11.1553,-1.82536
wpt_name,none,Zahanati,Kwa Mahundi,Zahanati Ya Nanyumbu,Shuleni
num_private,0,0,0,0,0


In [19]:
# Check shapes of each dataframe
df_train.shape, df_labels.shape, df_test.shape, df_merged.shape, target.shape

((59400, 40), (59400, 2), (14358, 40), (59400, 40), (59400,))

In [20]:
# Verifying merged dataframe feature info
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [21]:
# Adding 'train' column to training and test dataframes and adding value of 
# 1 if from training dataframe and 0 if from test dataframe
df_merged['train']=1
df_test['train']=0
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [22]:
# Verifying test dataframe feature info
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14358 entries, 0 to 14357
Data columns (total 41 columns):
id                       14358 non-null int64
amount_tsh               14358 non-null float64
date_recorded            14358 non-null object
funder                   13575 non-null object
gps_height               14358 non-null int64
installer                13570 non-null object
longitude                14358 non-null float64
latitude                 14358 non-null float64
wpt_name                 14358 non-null object
num_private              14358 non-null int64
basin                    14358 non-null object
subvillage               14264 non-null object
region                   14358 non-null object
region_code              14358 non-null int64
district_code            14358 non-null int64
lga                      14358 non-null object
ward                     14358 non-null object
population               14358 non-null int64
public_meeting           13573 non-null object
r

In [23]:
# Dataframe shape verification
df_merged.shape, df_test.shape

((59400, 41), (14358, 41))

In [24]:
# concatenating the train and test dateframes for munging
combined = pd.concat([df_merged, df_test])
combined.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1


In [25]:
# verifying dataframe feature info
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73758 entries, 0 to 14357
Data columns (total 41 columns):
id                       73758 non-null int64
amount_tsh               73758 non-null float64
date_recorded            73758 non-null object
funder                   69340 non-null object
gps_height               73758 non-null int64
installer                69315 non-null object
longitude                73758 non-null float64
latitude                 73758 non-null float64
wpt_name                 73758 non-null object
num_private              73758 non-null int64
basin                    73758 non-null object
subvillage               73293 non-null object
region                   73758 non-null object
region_code              73758 non-null int64
district_code            73758 non-null int64
lga                      73758 non-null object
ward                     73758 non-null object
population               73758 non-null int64
public_meeting           69639 non-null object
r

In [26]:
# Dropping features I do not feel fit the prediction model we will be using
combined.drop('construction_year',axis=1,inplace=True)
combined.drop('date_recorded',axis=1,inplace=True)
combined.drop('wpt_name',axis=1,inplace=True)
combined.drop('num_private',axis=1,inplace=True)
combined.drop('subvillage',axis=1,inplace=True)
combined.drop('region_code',axis=1,inplace=True)
combined.drop('ward',axis=1,inplace=True)
combined.drop('public_meeting',axis=1,inplace=True)
combined.drop('recorded_by',axis=1,inplace=True)
combined.drop('scheme_name',axis=1,inplace=True)
combined.drop('permit',axis=1,inplace=True)
combined.drop('extraction_type_group',axis=1,inplace=True)
combined.drop('extraction_type_class',axis=1,inplace=True)
combined.drop('management_group',axis=1,inplace=True)
combined.drop('payment',axis=1,inplace=True)
combined.drop('quality_group',axis=1,inplace=True)
combined.drop('quantity_group',axis=1,inplace=True)
combined.drop('source_type',axis=1,inplace=True)
combined.drop('source_class',axis=1,inplace=True)
combined.drop('waterpoint_type_group',axis=1,inplace=True)
combined.drop('installer',axis=1,inplace=True)
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73758 entries, 0 to 14357
Data columns (total 20 columns):
id                   73758 non-null int64
amount_tsh           73758 non-null float64
funder               69340 non-null object
gps_height           73758 non-null int64
longitude            73758 non-null float64
latitude             73758 non-null float64
basin                73758 non-null object
region               73758 non-null object
district_code        73758 non-null int64
lga                  73758 non-null object
population           73758 non-null int64
scheme_management    68942 non-null object
extraction_type      73758 non-null object
management           73758 non-null object
payment_type         73758 non-null object
water_quality        73758 non-null object
quantity             73758 non-null object
source               73758 non-null object
waterpoint_type      73758 non-null object
train                73758 non-null int64
dtypes: float64(3), int64(5), obj

In [27]:
# Factorizing the remaining categorical features
# factorize swaps unique categorical observations into a unique numeric
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined.district_code.head(5)

0    0
1    1
2    2
3    3
4    4
Name: district_code, dtype: int64

In [28]:
# Splitting the combined dataframe back into test and train using the 'train' feature we added above
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]

In [29]:
# Verifying the test dataframe 
test_df.head()

,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type,train
0,50785,0.0,162,1996,35.290799,-4.059696,4,2,6,37,321,8,5,8,1,0,3,1,3,0
1,51630,0.0,20,1569,36.656709,-3.309214,2,16,1,26,300,0,0,0,1,0,1,0,0,0
2,17168,0.0,-1,1567,34.767863,-5.004344,4,18,1,32,500,0,5,0,1,0,1,1,3,0
3,45559,0.0,143,267,38.058046,-9.418672,3,14,8,105,250,0,5,0,3,0,2,5,3,0
4,49871,500.0,1036,1260,35.006123,-10.950412,3,9,6,97,60,4,0,4,6,0,0,0,0,0


In [30]:
# checking to ensure the train dataframe has only training data 
train_df.train.value_counts()

1    59400
Name: train, dtype: int64

In [31]:
# checking to ensure the test dataframe has only test data 
test_df.train.value_counts()

0    14358
Name: train, dtype: int64

In [32]:
# verifying shapes are correct
train_df.shape, test_df.shape

((59400, 20), (14358, 20))

In [33]:
# Dropping the 'train' column we added above
train_df.drop(["train"], axis=1, inplace=True)
train_df.head()

C:\Users\Brian\Anaconda3\lib\site-packages\pandas\core\frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type
0,69572,6000.0,0,1390,34.938093,-9.856322,0,0,0,0,109,0,0,0,0,0,0,0,0
1,8776,0.0,1,1399,34.698766,-2.147466,1,1,1,1,280,1,0,1,1,0,1,1,0
2,34310,25.0,2,686,37.460664,-3.821329,2,2,2,2,250,0,0,0,2,0,0,2,1
3,67743,0.0,3,263,38.486161,-11.155298,3,3,3,3,58,0,1,0,1,0,2,3,1
4,19728,0.0,4,0,31.130847,-1.825359,1,4,4,4,0,-1,0,2,1,0,3,1,0


In [34]:
# verifying shapes are correct
train_df.shape

(59400, 19)

In [35]:
# Dropping the 'train' column we added above
test_df.drop(["train"], axis=1, inplace=True)
test_df.head()

,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type
0,50785,0.0,162,1996,35.290799,-4.059696,4,2,6,37,321,8,5,8,1,0,3,1,3
1,51630,0.0,20,1569,36.656709,-3.309214,2,16,1,26,300,0,0,0,1,0,1,0,0
2,17168,0.0,-1,1567,34.767863,-5.004344,4,18,1,32,500,0,5,0,1,0,1,1,3
3,45559,0.0,143,267,38.058046,-9.418672,3,14,8,105,250,0,5,0,3,0,2,5,3
4,49871,500.0,1036,1260,35.006123,-10.950412,3,9,6,97,60,4,0,4,6,0,0,0,0


In [36]:
# verifying shapes are correct
test_df.shape

(14358, 19)

In [37]:
#defining X as the entire train dataframe
X = train_df
# defining the target as 'y'
y = target

In [38]:
# Setting RendomForestClassifier estimators
model_rfc = RandomForestClassifier(n_estimators=1000)

In [39]:
# Setting cross validation score inputs
cross_val_score(model_rfc, X, y, cv=3)

array([0.80227273, 0.80373737, 0.79873737])

Large improvement with Random Forest model on prediction score over Logistic Regression 

In [40]:
# Fitting the Random Forest model
model_rfc.fit(X,y)
X.info() # Just printing the features to aid in matching features to score below
importances = model_rfc.feature_importances_
importances # list of feature scores, can match using above print out

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 19 columns):
id                   59400 non-null int64
amount_tsh           59400 non-null float64
funder               59400 non-null int64
gps_height           59400 non-null int64
longitude            59400 non-null float64
latitude             59400 non-null float64
basin                59400 non-null int64
region               59400 non-null int64
district_code        59400 non-null int64
lga                  59400 non-null int64
population           59400 non-null int64
scheme_management    59400 non-null int64
extraction_type      59400 non-null int64
management           59400 non-null int64
payment_type         59400 non-null int64
water_quality        59400 non-null int64
quantity             59400 non-null int64
source               59400 non-null int64
waterpoint_type      59400 non-null int64
dtypes: float64(3), int64(16)
memory usage: 9.1 MB


array([0.09803263, 0.03026977, 0.05628449, 0.07013578, 0.13460117,
       0.13185558, 0.01751376, 0.02366335, 0.02366318, 0.0338415 ,
       0.04867383, 0.01761533, 0.05641296, 0.01886318, 0.02555609,
       0.01333205, 0.11249576, 0.03092318, 0.05626639])

In [41]:
# Printing and sorting the most important features from Random Forest model
importances = model_rfc.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')

Feature ranking:
1. feature 4 (0.134601)
2. feature 5 (0.131856)
3. feature 16 (0.112496)
4. feature 0 (0.098033)
5. feature 3 (0.070136)
6. feature 12 (0.056413)
7. feature 2 (0.056284)
8. feature 18 (0.056266)
9. feature 10 (0.048674)
10. feature 9 (0.033841)
11. feature 17 (0.030923)
12. feature 1 (0.030270)
13. feature 14 (0.025556)
14. feature 7 (0.023663)
15. feature 8 (0.023663)
16. feature 13 (0.018863)
17. feature 11 (0.017615)
18. feature 6 (0.017514)
19. feature 15 (0.013332)
longitude, latitude, quantity, id, gps_height, extraction_type, funder, waterpoint_type, population, lga, source, amount_tsh, payment_type, region, district_code, management, scheme_management, basin, water_quality, 

In [42]:
# Random Forest model fitting
model_rfc.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [43]:
# Model set up for XGBoost classification model
model = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 4, maximize = False, eval_metric = 'merror', eta = .2,
                      max_depth = 14, colsample_bytree = .4)

In [44]:
# XGBoost fit and scoring
print(cross_val_score(model, X, y, cv=3))
model.fit(X,y)
importances = model.feature_importances_
importances
indices = np.argsort(importances)[::-1]

[0.80979798 0.80722222 0.80489899]


In [45]:
# Print the feature ranking without score
print("Feature ranking:")

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')

Feature ranking:
longitude, latitude, id, gps_height, population, funder, district_code, lga, region, source, amount_tsh, management, extraction_type, payment_type, basin, scheme_management, water_quality, waterpoint_type, quantity, 

In [46]:
# Printing feature ranking with rank and score
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

1. feature 4 (0.186400)
2. feature 5 (0.158845)
3. feature 0 (0.158817)
4. feature 3 (0.110183)
5. feature 10 (0.070847)
6. feature 2 (0.068500)
7. feature 8 (0.031206)
8. feature 9 (0.026018)
9. feature 7 (0.023002)
10. feature 17 (0.022755)
11. feature 1 (0.020719)
12. feature 13 (0.019946)
13. feature 12 (0.019705)
14. feature 14 (0.019064)
15. feature 6 (0.017499)
16. feature 11 (0.016379)
17. feature 15 (0.012615)
18. feature 18 (0.010579)
19. feature 16 (0.006920)


In [47]:
# Setting X_test initial value
X_test=test_df

In [48]:
# Verify head of X_test looks good
X_test.head()

,id,amount_tsh,funder,gps_height,longitude,latitude,basin,region,district_code,lga,population,scheme_management,extraction_type,management,payment_type,water_quality,quantity,source,waterpoint_type
0,50785,0.0,162,1996,35.290799,-4.059696,4,2,6,37,321,8,5,8,1,0,3,1,3
1,51630,0.0,20,1569,36.656709,-3.309214,2,16,1,26,300,0,0,0,1,0,1,0,0
2,17168,0.0,-1,1567,34.767863,-5.004344,4,18,1,32,500,0,5,0,1,0,1,1,3
3,45559,0.0,143,267,38.058046,-9.418672,3,14,8,105,250,0,5,0,3,0,2,5,3
4,49871,500.0,1036,1260,35.006123,-10.950412,3,9,6,97,60,4,0,4,6,0,0,0,0


In [49]:
# Run test dataframe through model to receive score
a=X_test['id']
#X_test.drop(['id'],axis=1, inplace=True)
y_pred = model.predict(X_test)

In [50]:
# Place prediction into dataframe
y_pred=pd.DataFrame(y_pred)
y_pred.head()

,0
0,non functional
1,functional
2,functional
3,non functional
4,functional


In [51]:
# adding 'id feature to dataframe'
y_pred['id']=a
y_pred.head()

,0,id
0,non functional,50785
1,functional,51630
2,functional,17168
3,non functional,45559
4,functional,49871


In [52]:
# Renaming columns in dataframe
y_pred.columns=['status_group','id']
y_pred.head()

,status_group,id
0,non functional,50785
1,functional,51630
2,functional,17168
3,non functional,45559
4,functional,49871


In [53]:
# Swapping columns in dataframe to fit the Kaggle submission requirements
y_pred=y_pred[['id','status_group']]
y_pred.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [54]:
# Verify shape of prediciton
y_pred.shape

(14358, 2)

In [55]:
# Output dataframe to CSV for Kaggle submission
pd.DataFrame(y_pred).to_csv("fifth_try.csv", index=False)

The XGBoost model was the best score, barely beating out the Random Forest model. 